In [2]:
import numpy as np
import math
from tabulate import tabulate

# Problem1

### i

#### $T(h) = \frac{h}{2}f(a) + h \sum_{i=1}^{n-1}f(a+ih) + \frac{h}{2}f(b)$

In [19]:
def trapezoidal(a,b,func,n):
    h = (b-a)/n
    fa = func(a)
    fb = func(b)
    T = 0
    
    for i in range(1,n):
        T = T + func(a+i*h)
    T = h/2*(fa+fb) + h*T
        
    return T

In [30]:
def Romberg(a,b,func,tol):
    err = math.inf
    h = b-a
    i = 1
    n = 1
    T = np.zeros((15,15))
    T[0][0] = trapezoidal(a,b,func,1)
    num = 0
    
    while err > tol:
        n = 2*n
        T[i][0] = trapezoidal(a,b,func,n)
        
        for k in range(1,i+1):
            T[i][k] = T[i][k-1] + (T[i][k-1] - T[i-1][k-1])/(4**k-1)
        
        err = abs((T[i][i-1] - T[i][i])/T[i][i])
        i = i+1
    
    T = T[:i,:i]
    
    for j in range(i):
        num += 2**j + 1
    
    return T,num

### ii

In [31]:
def func1(x):
    return x/(1+x**2)

def func2(x):
    return 1/(1-x)

def func3(x):
    return (1/math.sqrt(1-0.5*(math.sin(x))**2))

def func4(x):
    return (1/math.sqrt(1-0.8*(math.sin(x))**2))

def func5(x):
    return (1/math.sqrt(1-0.95*(math.sin(x))**2))

In [41]:
def trapezoidal_T(a,b,func,tol):
    rerr = math.inf
    h = b-a
    n = 1
    fa = func(a)
    fb = func(b)
    num = 0
    
    while rerr >= tol:
        T = 0
        for i in range(1,n):
            T = T + func(a+i*h)
        T = h/2*(fa+fb) + h*T
        
        num += n+1
        
        h = h/2
        n = 2*n
        T_ = 0
        for i in range(1,n):
            T_ = T_ + func(a+i*h)
        T_ = h/2*(fa+fb) + h*T_
        
        aerr = abs(T-T_)
        rerr = abs(T-T_)/abs(T_)
        
    return T_, num

In [42]:
def trapezoidal_S(a,b,func,tol):
    rerr = math.inf
    h = b-a
    n = 1
    fa = func(a)
    fb = func(b)
    num = 0
    
    while rerr >= tol:
        T1 = 0
        T2 = 0
        for i in range(n):
            T1 = T1 + func(a+(i+0.5)*h)
        for i in range(1,n):
            T2 = T2 + func(a+i*h)
            
        T = h/6*(fa+fb) + 2*h/3*T1 + h/3*T2
        
        num += 2*n -1
        
        h = h/2
        n = 2*n
        T1_ = 0
        T2_ = 0
        for i in range(n):
            T1_ = T1_ + func(a+(i+0.5)*h)
        for i in range(1,n):
            T2_ = T2_ + func(a+i*h)
        T_ = h/2*(fa+fb) + 2*h/3*T1_ + h/3*T2_
        
        aerr = abs(T-T_)
        rerr = abs(T-T_)/abs(T_)
        
    return T_, num

In [43]:
tol=1.e-6

In [46]:
def res_print(a,b,f,tol):
    (Tr,numr) = Romberg(a,b,f,tol)
    (Tt,numt) = trapezoidal_T(a,b,f,tol)
    (Ts,nums) = trapezoidal_S(a,b,f,tol)
    
    n = Tr.shape[0]
    hs = []
    h = b-a
    for i in range(n):
        hs.append(h)
        h = h/2
    info = {"h": hs, "Romberg Matrix": Tr}
    print(tabulate(info, headers='keys'))
    print()
    print("Romberg intergration:")
    print("The computed integral is ", Tr[-1][-1], " and the total number of function evaluation needed to compute is ", numr)
    print("Trapezoidal Rule:")
    print("The computed integral is ", Tt, " and the total number of function evaluation needed to compute is ", numt)
    print("Simpson's Rule:")
    print("The computed integral is ", Ts, " and the total number of function evaluation needed to compute is ", nums)

### $\int_0^3{\frac{x}{1+x^2}}dx = \frac{1}{2}log(10)$

In [47]:
res_print(0,3,func1,tol)

      h  Romberg Matrix
-------  -------------------------------------------------------------------
3        [0.45 0.   0.   0.   0.   0.  ]
1.5      [0.91730769 1.07307692 0.         0.         0.         0.        ]
0.75     [1.09700436 1.15690325 1.16249167 0.         0.         0.        ]
0.375    [1.13845857 1.15227663 1.15196819 1.15180115 0.         0.        ]
0.1875   [1.14811803 1.15133786 1.15127527 1.15126427 1.15126217 0.        ]
0.09375  [1.15050089 1.15129517 1.15129232 1.1512926  1.15129271 1.15129274]

Romberg intergration:
The computed integral is  1.1512927361761311  and the total number of function evaluation needed to compute is  69
Trapezoidal Rule:
The computed integral is  1.1512923533779378  and the total number of function evaluation needed to compute is  2058
Simpson's Rule:
The computed integral is  1.151293690906201  and the total number of function evaluation needed to compute is  524268


### $\int_0^{0.95}{\frac{1}{1-x}}dx = log(20)$

In [48]:
res_print(0,0.95,func2,tol)

        h  Romberg Matrix
---------  ------------------------------------------------------------------
0.95       [9.975 0.    0.    0.    0.    0.    0.   ]
0.475      [5.8922619  4.53134921 0.         0.         0.         0.
            0.        ]
0.2375     [4.08369332 3.48083712 3.41080298 0.         0.         0.
            0.        ]
0.11875    [3.35707585 3.11487002 3.09047222 3.0853876  0.         0.
            0.        ]
0.059375   [3.10177198 3.01667069 3.01012407 3.0088487  3.00854855 0.
            0.        ]
0.0296875  [3.0241335  2.99825401 2.99702623 2.99681833 2.99677115 2.99675964
            0.        ]
0.0148437  [3.00299624 2.99595049 2.99579692 2.99577741 2.99577333 2.99577235
            2.99577211]

Romberg intergration:
The computed integral is  2.9957721108751363  and the total number of function evaluation needed to compute is  134
Trapezoidal Rule:
The computed integral is  2.9957327207096607  and the total number of function evaluation needed to comp

### $\int_0^{\pi/2}{\frac{1}{\sqrt{1-msin^2(x)}}}dx, m=0.5$

In [49]:
res_print(0,math.pi/2,func3,tol)

        h  Romberg Matrix
---------  --------------------------------------------------------
1.5708     [1.8961189 0.        0.        0.        0.       ]
0.785398   [1.85495913 1.84123921 0.         0.         0.        ]
0.392699   [1.85407523 1.85378059 1.85461669 0.         0.        ]
0.19635    [1.85407468 1.85407449 1.85409409 1.85408579 0.        ]
0.0981748  [1.85407468 1.85407468 1.85407469 1.85407438 1.85407434]

Romberg intergration:
The computed integral is  1.8540743368882955  and the total number of function evaluation needed to compute is  36
Trapezoidal Rule:
The computed integral is  1.8540746773016668  and the total number of function evaluation needed to compute is  10
Simpson's Rule:
The computed integral is  1.8540758828212553  and the total number of function evaluation needed to compute is  2097130


### $\int_0^{\pi/2}{\frac{1}{\sqrt{1-msin^2(x)}}}dx, m=0.8$

In [50]:
res_print(0,math.pi/2,func4,tol)

        h  Romberg Matrix
---------  --------------------------------------------------------
1.5708     [2.54160185 0.         0.         0.         0.        ]
0.785398   [2.28474559 2.19912684 0.         0.         0.        ]
0.392699   [2.25762153 2.24858017 2.25187706 0.         0.        ]
0.19635    [2.25720546 2.25706677 2.25763255 2.2577239  0.        ]
0.0981748  [2.25720533 2.25720528 2.25721452 2.25720788 2.25720586]

Romberg intergration:
The computed integral is  2.2572058568489357  and the total number of function evaluation needed to compute is  36
Trapezoidal Rule:
The computed integral is  2.257205326820873  and the total number of function evaluation needed to compute is  19
Simpson's Rule:
The computed integral is  2.2572069427277994  and the total number of function evaluation needed to compute is  2097130


### $\int_0^{\pi/2}{\frac{1}{\sqrt{1-msin^2(x)}}}dx, m=0.95$

In [51]:
res_print(0,math.pi/2,func5,tol)

        h  Romberg Matrix
---------  -------------------------------------------------------------------
1.5708     [4.29780553 0.         0.         0.         0.         0.        ]
0.785398   [3.23285521 2.87787177 0.         0.         0.         0.        ]
0.392699   [2.94266735 2.84593806 2.84380914 0.         0.         0.        ]
0.19635    [2.90897328 2.89774192 2.90119551 2.90210641 0.         0.        ]
0.0981748  [2.90833756 2.90812566 2.90881791 2.9089389  2.90896569 0.        ]
0.0490874  [2.90833725 2.90833714 2.90835124 2.90834384 2.9083415  2.90834089]

Romberg intergration:
The computed integral is  2.9083408922712546  and the total number of function evaluation needed to compute is  69
Trapezoidal Rule:
The computed integral is  2.908337248444657  and the total number of function evaluation needed to compute is  36
Simpson's Rule:
The computed integral is  2.908339980915769  and the total number of function evaluation needed to compute is  2097130


# Problem2

### i.

#### $\hat{f}(x) = f(x) + \delta(x)$ where $|\delta(x) |\leq \delta$ and $T(h) = \frac{h}{2}f(a) + h \sum_{i=1}^{n-1}f(a+ih) + \frac{h}{2}f(b)$
#### When inexact function evaluations are used, $T(h) = \frac{h}{2}[f(a)+\delta(a)] + h \sum_{i=1}^{n-1}[f(a+ih)+\delta(a+ih)] + \frac{h}{2}[f(b)+\delta(b)]$
#### $|\int_a^b{f(x)}dx - T(h)|$
#### $= |\int_a^b{f(x)}dx - \{\frac{h}{2}[f(a)+\delta(a)] + h \sum_{i=1}^{n-1}[f(a+ih)+\delta(a+ih)] + \frac{h}{2}[f(b)+\delta(b)]\}|$
#### $\leq |\int_a^b{f(x)}dx - [\frac{h}{2}f(a) + h \sum_{i=1}^{n-1}f(a+ih) + \frac{h}{2}f(b)]| + |\frac{h}{2}\delta(a) + h \sum_{i=1}^{n-1}\delta(a+ih) + \frac{h}{2}\delta(b)|$
#### $\leq \frac{b-a}{12}h^2 max_{[a,b]}{|f''(x)|} + nh\delta$
#### $= \frac{b-a}{12}h^2 max_{[a,b]}{|f''(x)|} + (b-a)\delta$

### ii.

#### Instead of $f(x) = x^{5/2}$, we use $\hat{h}(x) = x^{5/2} + 0.01r(x)$

In [15]:
def func(x):
    return x**(5/2)

In [16]:
def trapezoidal_T(a,b,func,tol):
    rerr = math.inf
    h = b-a
    n = 1
    fa = func(a)
    fb = func(b)
    h_p = []
    T_p = []
    rerr_p = []
    n_p = []
    
    while rerr >= tol:
        x = np.linspace(a, b, n+1)
        Ta = func(x) + 0.01 * np.random.randn(1, n+1)
        T = h * np.sum(Ta)- h/2*(fa+fb)
        
        h = h/2
        n = 2*n
        x = np.linspace(a, b, n+1)
        Ta_ = func(x) + 0.01 * np.random.randn(1, n+1)
        T_ = h * np.sum(Ta_)- h/2*(fa+fb)
        
        aerr = abs(T-T_)
        rerr = abs(T-T_)/abs(T_)
        
        n_p.append(n)
        h_p.append(h)
        T_p.append(T_)
        rerr_p.append(rerr)
    
    info = {'n': n_p, 'h/2': h_p, 'T(h/2)': T_p, '|T(h)-T(h/2)|/|T(h/2)|': rerr_p}
    print(tabulate(info, headers='keys'))

In [18]:
trapezoidal_T(0,1,func,1.e-6)

       n          h/2    T(h/2)    |T(h)-T(h/2)|/|T(h/2)|
--------  -----------  --------  ------------------------
       2  0.5          0.324912               0.502184
       4  0.25         0.302901               0.145075
       8  0.125        0.289021               0.00865121
      16  0.0625       0.287902               0.0134553
      32  0.03125      0.284277               0.00485818
      64  0.015625     0.284564               0.0106843
     128  0.0078125    0.285576               0.0055625
     256  0.00390625   0.286349               0.00500864
     512  0.00195312   0.285737               0.00125452
    1024  0.000976562  0.28574                0.00164163
    2048  0.000488281  0.285768               0.000738615
    4096  0.000244141  0.285784               0.000223709
    8192  0.00012207   0.285664               1.94175e-05
   16384  6.10352e-05  0.285837               0.000993982
   32768  3.05176e-05  0.285702               6.3564e-05
   65536  1.52588e-05  0.285674 

# Problem3

In [13]:
def func(x,y):
    return math.exp(-x*y)

### i.  $\Omega=\{ (x,y) ∈ R2:x,y∈ [0,1]\}$, the unit square

In [14]:
def double_trapezoidal(x0,xn,y0,yn,nx,ny):
    #calcaulte h for x,y
    hx = (xn-x0)/nx
    hy = (yn-y0)/ny
    F = np.zeros((nx+1,ny+1))
    
    #generate matrix F with function value for each pair of x,y
    for j in range(ny+1):
        for i in range(nx+1):
            F[i][j] = func(x0+i*hx, y0+j*hy)
    
    #calculate the sum of entries in four boudaries
    res1 = 0
    for i in range(1,nx):
        res1 += F[i][0] + F[i][-1]
    for j in range(1,ny):
        res1 += F[0][j] + F[-1][j]
    
    #calculate the sum of the rest entries
    F_ = F[1:nx,1:ny]
    res2 = 0
    for i in range(nx-1):
        for j in range(ny-1):
            res2 += F_[i][j]
            
    #calculate the integral with 3 kind of values
    integral = hx*hy/4*((F[0][0] + F[0][-1] + F[-1][0] + F[-1][-1]) + 2*res1 + 4*res2)
    
    #calculate the number of function value required
    tn = (nx+1)*(ny+1)

    return integral, tn

In [15]:
x0=0
y0=0
xn=1
yn=1
for nx in (20,50,100):
    for ny in (20,50,100):
        (res, tn) = double_trapezoidal(x0,xn,y0,yn,nx,ny)
        print("For nx = ",nx,", ny = ", ny, ": ")
        print("The approximate value of the integral obtained by my approach is: ", res)
        print(" and the number of function value required is: ", tn)
        print()

For nx =  20 , ny =  20 : 
The approximate value of the integral obtained by my approach is:  0.7966978731970396
 and the number of function value required is:  441

For nx =  20 , ny =  50 : 
The approximate value of the integral obtained by my approach is:  0.7966565800926085
 and the number of function value required is:  1071

For nx =  20 , ny =  100 : 
The approximate value of the integral obtained by my approach is:  0.7966506809411552
 and the number of function value required is:  2121

For nx =  50 , ny =  20 : 
The approximate value of the integral obtained by my approach is:  0.7966565800926095
 and the number of function value required is:  1071

For nx =  50 , ny =  50 : 
The approximate value of the integral obtained by my approach is:  0.796615317606477
 and the number of function value required is:  2601

For nx =  50 , ny =  100 : 
The approximate value of the integral obtained by my approach is:  0.7966094228294618
 and the number of function value required is:  5151

### ii.  $\Omega=\{ (x,y) ∈ R2:x^2+y^2 \leq 1, x,y\geq0\}$, the quarter of the unit disk lying in the first quadrant.

In [16]:
def double_trapezoidal2(x0,xn,y0,yn,nx,ny):
    #calcaulte h for x,y
    hx = (xn-x0)/nx
    hy = (yn-y0)/ny
    F = np.zeros((nx+1,ny+1))
    
    #generate matrix F with function value for each pair of x,y while y depends on value of x
    for i in range(nx+1):
        ymax = math.sqrt(1-(i*hx)**2)
        for j in range(ny+1):
            if j*hy <= ymax:
                F[i][j] = func(x0+i*hx, y0+j*hy)
    
    #calculate the sum of entries in four boudaries
    res1 = 0
    for i in range(1,nx):
        res1 += F[i][0] + F[i][-1]
    for j in range(1,ny):
        res1 += F[0][j] + F[-1][j]
      
    #calculate the sum of the rest entries
    F_ = F[1:nx,1:ny]
    res2 = 0
    for i in range(nx-1):
        for j in range(ny-1):
            res2 += F_[i][j]
            
    #calculate the integral with 3 kind of values       
    integral = hx*hy/4*((F[0][0] + F[0][-1] + F[-1][0] + F[-1][-1]) + 2*res1 + 4*res2)
    
    #calculate the number of funciton value required
    tn = 0
    for i in range(nx+1):
        for j in range(ny+1):
            if F[i][j]!=0:
                tn += 1

    return  integral,tn

In [17]:
x0=0
y0=0
xn=1
yn=1
for nx in (20,50,100):
    for ny in (20,50,100):
        (res,tn) = double_trapezoidal2(x0,xn,y0,yn,nx,ny)
        print("For nx = ",nx,", ny = ", ny, ": ")
        print("The approximate value of the integral obtained by my approach is: ", res)
        print(" and the number of function value required is: ", tn)
        print()

For nx =  20 , ny =  20 : 
The approximate value of the integral obtained by my approach is:  0.6700897026340855
 and the number of function value required is:  333

For nx =  20 , ny =  50 : 
The approximate value of the integral obtained by my approach is:  0.6698171322510547
 and the number of function value required is:  815

For nx =  20 , ny =  100 : 
The approximate value of the integral obtained by my approach is:  0.6712998382003065
 and the number of function value required is:  1623

For nx =  50 , ny =  20 : 
The approximate value of the integral obtained by my approach is:  0.6704359156428613
 and the number of function value required is:  816

For nx =  50 , ny =  50 : 
The approximate value of the integral obtained by my approach is:  0.6737805827976127
 and the number of function value required is:  2011

For nx =  50 , ny =  100 : 
The approximate value of the integral obtained by my approach is:  0.6744755023607294
 and the number of function value required is:  4000


# Problem4

#### $T(h) = \frac{h}{2}f(a) + h \sum_{i=1}^{n-1}f(a+ih) + \frac{h}{2}f(b)$

#### $S(h)=\frac{h}{6}f(a) + \frac{2h}{3} \sum_{i=0}^{n-1}f(a+(i+\frac{1}{2}h)) + \frac{h}{3} \sum_{i=1}^{n-1}f(a+ih) + \frac{f}{6}f(b)$

#### Since we know $T_{0,0} = T(h) = \frac{h}{2}f(a) + h \sum_{i=1}^{n-1}f(a+ih) + \frac{h}{2}f(b)$ and $T_{1,0} = T(\frac{h}{2}) = \frac{h}{4}f(a) + \frac{h}{2} \sum_{i=1}^{n-1}f(a+i\frac{h}{2}) + \frac{h}{4}f(b)$
#### Then we get:
#### $T_{1,1} = T_{1,0} + \frac{T_{1,0} - T_{0,0}}{4-1}$
#### $= \frac{4}{3}T_{1,0} - \frac{1}{3}T_{0,0} $
#### $= \frac{h}{3}f(a) + \frac{2h}{3} \sum_{i=1}^{n-1} f(a + i\frac{h}{2}) + \frac{h}{3}f(b) - \frac{h}{6}f(a) - \frac{h}{3} \sum_{i=1}^{n-1}f(a+ih) - \frac{h}{6}f(b)$
#### $= \frac{h}{6}f(a) + \frac{2h}{3} \sum_{i=1}^{n-1} f(a + i\frac{h}{2}) - \frac{h}{3} \sum_{i=1}^{n-1}f(a+ih) + \frac{h}{6}f(b)$
#### $= S(h)$